#  SLAM algorithm step by step
This notebooks allows to perform step-by-step matching of the scans registered in a small apartment.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import sys
sys.path.append('../')

import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


from src.grid_maps import log_odds, prob
from src.scan_processing import scan2xy
import src.feature_detection as features
import src.hc as hc
import src.icp as icp
import src.graphslam as gs

import src.grid_maps as g

from tqdm.notebook import tqdm
from PIL import Image, ImageDraw

Before starting the notebook it is needed to preprocess the scans and saved odomtery data using `prepare_scans.py` script.
The script:
- groups scans taken in same places
- removes scans taken while moving
- estimates robot poses based on saved odometry or controls

```
python ..\src\prepare_scans.py
```

The results are stored in `mapping.pickle`.

In [ ]:
with open('mapping.pickle', 'rb') as f:
    (scans, poses, controls) = pickle.load(f)

# combine poses into a single array
poses = np.vstack(poses)

# remove the first scan (usually taken in a static position before the procedure's start)
scans = scans[1:]
poses_odo = poses[1:]
controls = controls[1:]

# change controls
controls = np.vstack(controls)
controls[:,0] = controls[:,0]*0.7 # to account for lower robot speed
controls[:,1] = controls[:,1] *1

# estimate robot poses based on the controls
poses_odo = []
posodo = np.r_[0,0,0]

for c in controls:
    posodo = posodo + np.r_[np.cos(posodo[2])*c[0], np.sin(posodo[2])*c[0], c[1]]
    poses_odo.append(posodo)
    
poses_odo = np.vstack(poses_odo)


### SLAM Parameters
The SLAM algorithm takes parameters governing its steps - line extraction from the scans, association thresholds for ICP points etc. The detailed explanation can be found in the paper:

**M. Kolakowski, “Automated Calibration of RSS Fingerprinting Based Systems Using a Mobile Robot and Machine Learning”, Sensors , vol. 21, 6270, Sep. 2021 https://doi.org/10.3390/s21186270**

In [ ]:
sp = {
    "extr_angle_range": np.pi / 3,
    "extr_split_th": 0.1,
    "extr_min_len": 0.6,
    "extr_min_points": 10,

    "mrg_max_dist": -0.2,
    "mrg_a_tol": 0.1,
    "mrg_b_tol": 0.1,
    "mrg_fit_tol": 0.1,

    "association_th": [0.3],

    "an_th": 0.3,
    "d_th": 0.2,
    "corr_points_th": 0.1

}

### ICP matching of consecutive scans
The scans are consecutively matched to each other to obtain a rough map of the entire apartment for the first GraphSLAM iteration.

In [ ]:
transformations = []
fig, axs = plt.subplots(int(len(scans)/8+1), 8, figsize=(24, 3*int(len(scans)/8+1)))

heading = 0

progress_bar = tqdm(range(len(scans)))

for i, ax in zip(range(len(scans)-1), axs.ravel()[:]):
#     clear_output(wait=True)
#     print(i)
    s1 = scans[i]
    s2 = scans[i+1]
    c = controls[i+1]
    ic = np.r_[np.cos(heading)*c[0], np.sin(heading)*c[0], c[1]]
    H = icp.transformation_matrix(ic[2], ic[:2])
    ic = np.r_[H[:2, 2], c[1]]
#     print(ic)
    try:
        t = icp.match_scans(s1,s2, ic, sp, iters=5)
    except:
        print("Error for scan {}".format(i))
        t = np.r_[1,1,1]*np.nan
        
    transformations.append(t)
    Hi = icp.transformation_matrix(ic[2], ic[:2])
    
    
    H = hc.translation(t[:2]).dot(hc.rotation(t[2]))

    s1h = hc.ec_hc(s1[:,:2])
    s2h = hc.ec_hc(s2[:,:2])

    s2t = H.dot(s2h)
    s2i = Hi.dot(s2h)

    ax.set_title(i)
    ax.set_aspect('equal')
    ax.scatter(s1.T[0], s1.T[1], s=3)
    ax.scatter(s2i[0], s2i[1], marker='.', s=1)
    ax.scatter(s2t[0], s2t[1], marker='.', s=1)
    ax.grid()
    progress_bar.update(1)

plt.show()

In [ ]:
with open('icp_trans_mapping.pickle', 'wb') as f:
    pickle.dump(transformations, f)

Check if all of the ICP transformations were computed. If any row is `False` repeat the fitting procedure for the given scan pair using different parameters.

In [ ]:
np.argwhere(np.isnan(transformations).all(axis=1))

In [ ]:
sp = {
    "extr_angle_range": np.pi / 3,
    "extr_split_th": 0.1,
    "extr_min_len": 0.3,
    "extr_min_points": 10,

    "mrg_max_dist": -0.2,
    "mrg_a_tol": 0.1,
    "mrg_b_tol": 0.1,
    "mrg_fit_tol": 0.1,

    "association_th": [0.3],

    "an_th": 0.9,
    "d_th": 0.9,
    "corr_points_th": 0.4

}

In [ ]:
heading=0
for i in [77]:

    s1 = scans[i]
    s2 = scans[i+1]
    c = controls[i+1]
    ic = np.r_[np.cos(heading)*c[0], np.sin(heading)*c[0], c[1]]
    H = icp.transformation_matrix(ic[2], ic[:2])
    ic = np.r_[H[:2, 2], c[1]]

    try:
        t = icp.match_scans(s1,s2, ic, sp, iters=7)
    except:
        print("Error for scan {}".format(i))
        t = np.r_[1,1,1]*np.nan
        
    transformations[i] = t
    Hi = icp.transformation_matrix(ic[2], ic[:2])
    
    
    H = hc.translation(t[:2]).dot(hc.rotation(t[2]))

    s1h = hc.ec_hc(s1[:,:2])
    s2h = hc.ec_hc(s2[:,:2])

    s2t = H.dot(s2h)
    s2i = Hi.dot(s2h)
    
    plt.title(i)
    plt.axes().set_aspect('equal')
    plt.scatter(s1.T[0], s1.T[1], s=3)
    plt.scatter(s2i[0], s2i[1], marker='.', s=1)
    plt.scatter(s2t[0], s2t[1], marker='.', s=1)
    plt.grid()

    plt.show()

Save the complete transformations list

In [ ]:
with open('icp_trans_mapping2.pickle', 'wb') as f:
    pickle.dump(transformations, f)

### Visualize initial estimate of poses

In [ ]:
transformations = pickle.load(open('icp_trans_mapping2.pickle', 'rb'))

In [ ]:
poses = []
heading = 0
pose = np.r_[0,0,0]
poses.append(pose)
for t in transformations[:]:

    heading = poses[-1][2] + t[2]
    px = poses[-1][0] +  t[0]* np.cos(heading)- t[1]* np.sin(heading)
    py = poses[-1][1] +  t[1]* np.cos(heading)+ t[0]* np.sin(heading)
    ph = poses[-1][2] + t[2]
    pose = np.r_[px, py, ph]
    
    poses.append(pose)
poses = np.vstack(poses)

plt.figure()
plt.axes().set_aspect('equal')
plt.plot(poses.T[0],poses.T[1], marker='o' ,label='icp matching')
plt.plot(poses_odo.T[0], poses_odo.T[1], marker='o', label='odometry')
plt.legend()

## GraphSLAM 

### Graph initialization
The graph is initialized with poses resulting from the odometry measurements. The edges are the results of the consecutive scans ICP fitting.

In [ ]:
sp = {
    "extr_angle_range": np.pi / 3,
    "extr_split_th": 0.2,
    "extr_min_len": 0.6,
    "extr_min_points": 10,

    "mrg_max_dist": -0.2,
    "mrg_a_tol": 0.1,
    "mrg_b_tol": 0.1,
    "mrg_fit_tol": 0.1,

    "association_th": [0.4],

    "an_th": 0.3,
    "d_th": 0.2,
    "corr_points_th": 0.06

}

In [ ]:
# initialize graph
graph = gs.Graph()
graph.init_nodes(poses_odo)
graph.init_edges()

In [ ]:
for t, i in zip(transformations, range(len(transformations))):
    graph.add_icp_edge(i, i+1, t)

In [ ]:
x = gs.run_graphSlam(graph)

In [ ]:
graph.update_graph(x)

In [ ]:
graph.plot()

In [ ]:
pickle.dump(graph, open('graph1.pickle', 'wb'))

Now it is time to find constraints between the non consecutive scans. A single iteration is implemented using `graphSLAM_iter` function.

In [ ]:
def graphSLAM_iter(graph, max_distance, min_common_map, scans):
    icp_candidates = gs.find_icp_candidates(graph, max_distance, min_common_map, scans)
    print('Starting graphSLAM for max_distance: {} and min_common_map: {}'.format(max_distance, min_common_map))
    i = 1
    while len(icp_candidates)>0:
        print('Iteration {}, found {} scan pairs for ICP'.format(i, len(icp_candidates)))
       
        # get transformations
        print('getting ICP edges...')
        progress_bar = tqdm(range(len(icp_candidates)))

        gs.create_icp_edges(graph, scans, icp_candidates, sp,progress_bar)

        # run graphslam
        print('optimizing graph...' )
        x = gs.run_graphSlam(graph)
        graph.update_graph(x)
        graph.plot()

        # dump
        pickle.dump(graph, open('graph.pickle', 'wb'))

        # check if there are other candidates
        icp_candidates = gs.find_icp_candidates(graph, max_distance, min_common_map, scans)

    print('No more new edges for the parameters. Change max_distance and min_common_map and run again.')
    
    
    
    
    return graph

In [ ]:
def show_gridmap(graph):
    
    xrs = graph.get_poses()

    size= 20
    res= 0.04
    gridmap = g.init_gridmap(size, res)

    for s, p in zip(scans[:49], xrs[:49]):
        H = hc.translation(p[:2]).dot(hc.rotation(p[2]))
        st = H.dot(hc.ec_hc(s[:,:2]))
    #     r = np.linalg.norm(st[:,:2]-p[:2], axis=1)
        gmap = g.points2gridmap(size, res, p, st)

        gridmap = g.merge_maps(gridmap, gmap)


    for s, p in zip(scans[51:], xrs[51:]):
        H = hc.translation(p[:2]).dot(hc.rotation(p[2]))
        st = H.dot(hc.ec_hc(s[:,:2]))
    #     r = np.linalg.norm(st[:,:2]-p[:2], axis=1)
        gmap = g.points2gridmap(size, res, p, st)

        gridmap = g.merge_maps(gridmap, gmap)


    plt.figure(figsize=(10,10))
    plt.imshow(gridmap, cmap='bone_r')
    
    return gridmap

#### First iteration

In [ ]:
graph = graphSLAM_iter(graph, 1, 0.8, scans)

In [ ]:
show_gridmap(graph)

In [ ]:
graph = graphSLAM_iter(graph, 1, 0.75, scans)

In [ ]:
graph = graphSLAM_iter(graph, 1, 0.7, scans)

In [ ]:
show_gridmap(graph)

In [ ]:
graph = graphSLAM_iter(graph, 1.5, 0.75, scans)

In [ ]:
show_gridmap(graph)

In [ ]:
graph = graphSLAM_iter(graph, 1.5, 0.7, scans)

In [ ]:
graph = graphSLAM_iter(graph, 2, 0.6, scans)

In [ ]:
show_gridmap(graph)

## Final gridmap

In [ ]:
def center_scans(scans, scan_points):
    """Center the scans so that the resulting gridmap will be as small as possible

    Parameters
    ----------
    scans: list
        scans [x,y] format
    scan_points: ndarray
        scan locations
    trans: ndarray
        values which was subtracted from the points 

    Returns
    -------
    centered: list
        centered scans
    scans_locs: ndarray
        updated scan locations
    med: ndarray
        transformation, which was applied (the value should be subtracted from the x-y coordinates if there's a need to
        transform other points
    """
    min = np.min(scan_points, axis=0)
    max = np.max(scan_points, axis=0)
    med = (min + max) / 2
    centered = []
    for s in scans:
        centered.append(s - med)

    scan_locs = scan_points - med
    return centered, scan_locs, med

Transform scan points according to the computed poses

In [ ]:
xrs = graph.get_poses()
gridmap = g.init_gridmap(size, res)
scans_trans = []
for s, p in zip(scans, xrs):
    H = hc.translation(p[:2]).dot(hc.rotation(p[2]))
    st = hc.hc_ec(H.dot(hc.ec_hc(s[:,:2])))
    scans_trans.append(st)
    

Center scans and compute shape

In [ ]:
centered, scan_locs, med = center_scans(scans_trans,np.vstack(xrs)[:,:2])

all_points = np.vstack(centered)
size = tuple(e for e in np.abs(all_points).max(axis=0) * 2+ 0.5)

Set gridmap parameters

In [ ]:
# gridmap resolution
gridmap_res = 0.05

# gridmap size override, which might be needed in case of some outlier points
size = (11,10)

Create gridmaps

In [ ]:
gridmaps=[]
for s, p, angle in zip(centered, scan_locs, np.vstack(xrs)[:,2]):
    gmap = g.points2gridmap(size, gridmap_res, np.r_[p, angle], hc.ec_hc(s) )
    gridmaps.append(gmap)

Merge into one map

In [ ]:
gmap = gridmaps[0]

for gr in gridmaps[1:]:
    gmap = g.merge_maps(gmap, gr)

Show map

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(gmap, cmap='bone_r')

Create an image using PIL and save

In [ ]:
# convert to an image
img = Image.fromarray(np.uint8((1-gmap) * 255) , 'L')
img = img.convert('RGB')

# save image
img.save('map.jpg')